NTUST course: Computer Vision and Applications (CI5336701, 2022 Spring).  
Final Project: Reconstruct 3D from stereoscopic images and colorize 3D point cloud.  
Date Due: 2022. Jun. 13th, PM11:55 (around three weeks) Description:
1. Write programs for reconstructing 3D points from two images, then, output a “Color” .xyz file. (choose your tools, ex. C++/C, python, openCV, Matlab).
2. The intrinsic and extrinsic parameters of both images are given in CalibrationData.txt. In this project, you need to write programs for importing both image sequences, and analyzing the both synchronized images. A fundamental matrix is given for assisting you to find the corresponding features in left and right images. Once corresponding features are determined, please calculate their 3D, then store them as a ID.xyz. Please reject all outliers by verifying their reprojection error.
3. An extra color texture file, called TextureImage.JPG, was taken at different device. You may need it to colorize your ID.xyz file. Please use one 3D tool, such as Meshlab, to retrieve 3D point coordinate and use image tool, such photoshop, to obtain 2D coordinate. Therefore, you can collect 2D-3D corresponding points to estimate the camera project matrix. Based on project matrix, try to project “Color” on the .xyz file. Then, save .xyz file again with color. A color .xyz file should be the format as
[X Y Z R G B], in each line of a text file with .xyz extension.
4. Please write a short report (upto 3 pages, A4), and use Meshlab or other 3D viewer to verify
your result.
5. Deliverable: There are three types of data you should provide:
1) Source code in C++/C, python or Matlab, with simple comment. (single or two programs with following a and b function)
a. To reconstruct 3D point in .xyz file.
b. To colorize 3D point and export as a color .xyz file
2) (Optional) Execution file (.exe), if you use c/c++ code, please build it into exe. 3) Result of color 3D point data (stored as ID.xyz)
4) PDF: description document (A4 within 3 pages).
Please zip all your files, then, upload on moodle2 (http://moodle2.ntust.edu.tw/) by due 6/13 PM11:55. Evaluation rule: 1. 80 % correct 3D point. (if you reject outliers/noise, it will be better)
2. 20 % correct “Color” 3D point cloud Note: This assignment will occupy 25% of final grade.
Hint: The 3D model is roughly 130 mm in height.

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
from google.colab import drive
import os
np.set_printoptions(suppress=True) # 不以科學記號顯示數值
drive.mount('/content/drive')
#Hw4_Image_Ori_Path = '/content/drive/MyDrive/Course/11002/Computer Vision/HW4/toCalibrated.jpg'
#Hw4_Image_Result_Path = "/content/drive/MyDrive/Course/11002/Computer Vision/HW4/M11015Q03.jpg"
#Ori_Imgae = cv2.imread(Hw4_Image_Ori_Path)
#cv2_imshow(Ori_Imgae)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Left Camera Intrinsic parameter
CameraIntrinsic_Left = np.asarray([
[1035.278669095568, 0.000000000000, 295.500377771516],
[0.000000000000, 1034.880664685675, 598.224722223280],
[0.000000000000, 0.000000000000, 1.000000000000]
])

#Right Camera Intrinsic parameter
CameraIntrinsic_Right = np.asarray([
[1036.770200759934, 0.000000000000, 403.040387412710],
[0.000000000000, 1037.186415753241, 612.775486819306],
[0.000000000000, 0.000000000000, 1.000000000000],
])

#Left Camera Extrinsic parameter
CameraExtrinsic_Left = np.asarray([
[1.000000000000, 0.000000000000, 0.000000000000, 0.000000000000],
[0.000000000000, 1.000000000000, 0.000000000000, 0.000000000000],
[0.000000000000, 0.000000000000, 1.000000000000, 0.000000000000],
])

#Right Camera Extrinsic parameter
CameraExtrinsic_Right = np.asarray([
[0.958173249509, 0.009400631103, 0.286034354684, -69.855978076557],
[-0.009648701145, 0.999953303246, -0.000542119475, 0.110435878514],
[-0.286026094074, -0.002240415626, 0.958219209809, 14.517584144224],
])

# Lecture 06 P.30 P = K[R|t]
L_P = np.dot(CameraIntrinsic_Left, CameraExtrinsic_Left)
R_P = np.dot(CameraIntrinsic_Right, CameraExtrinsic_Right)

#FMatrix
FMatrix = np.asarray([
[-0.000000076386, 0.000003333642, -0.001745446202],
[0.000000105635, -0.000000103805, -0.011352133262],
[-0.000285872783, 0.010010406460, 1.000000000000]
])

In [ ]:
IMG_DIR = "./SidebySide/"
VIS_DIR = "./feature_points_visualization/"
IMG_SIDE_L = '/content/drive/MyDrive/Course/11002/Computer Vision/FinalProject/L'
IMG_SIDE_R = '/content/drive/MyDrive/Course/11002/Computer Vision/FinalProject/R'
IMG_CONCAT = '/content/drive/MyDrive/MyDrive/Course/11002/Computer Vision/FinalProject/ConcatImage'

if not os.path.isdir(VIS_DIR):
    os.makedirs(VIS_DIR, exist_ok=True)
if not os.path.isdir(IMG_DIR):
    os.makedirs(IMG_DIR, exist_ok=True)
  

In [ ]:
from os import walk
img_L = []
for (dirpath, dirnames, filenames) in walk(IMG_SIDE_L):
    img_L.extend(filenames)
    break
print(img_L)
img_R = []
for (dirpath, dirnames, filenames) in walk(IMG_SIDE_R):
    img_R.extend(filenames)
    break
print(img_R)

['L175.JPG', 'L161.JPG', 'L149.JPG', 'L072.JPG', 'L270.JPG', 'L264.JPG', 'L066.JPG', 'L099.JPG', 'L106.JPG', 'L112.JPG', 'L258.JPG', 'L259.JPG', 'L271.JPG', 'L265.JPG', 'L197.JPG', 'L183.JPG', 'L154.JPG', 'L011.JPG', 'L159.JPG', 'L005.JPG', 'L165.JPG', 'L171.JPG', 'L140.JPG', 'L039.JPG', 'L213.JPG', 'L168.JPG', 'L207.JPG', 'L206.JPG', 'L004.JPG', 'L158.JPG', 'L038.JPG', 'L010.JPG', 'L170.JPG', 'L212.JPG', 'L164.JPG', 'L172.JPG', 'L199.JPG', 'L166.JPG', 'L006.JPG', 'L238.JPG', 'L013.JPG', 'L210.JPG', 'L211.JPG', 'L239.JPG', 'L012.JPG', 'L205.JPG', 'L007.JPG', 'L204.JPG', 'L198.JPG', 'L167.JPG', 'L188.JPG', 'L017.JPG', 'L177.JPG', 'L163.JPG', 'L003.JPG', 'L229.JPG', 'L201.JPG', 'L173.JPG', 'L215.JPG', 'L214.JPG', 'L189.JPG', 'L162.JPG', 'L002.JPG', 'L016.JPG', 'L228.JPG', 'L200.JPG', 'L176.JPG', 'L148.JPG', 'L174.JPG', 'L160.JPG', 'L028.JPG', 'L216.JPG', 'L217.JPG', 'L001.JPG', 'L203.JPG', 'L202.JPG', 'L015.JPG', 'L029.JPG', 'L014.JPG', 'L000.JPG', 'L145.JPG', 'L231.JPG', 'L219.JPG', 'L2

In [ ]:
for img in img_L:
  img_R = img.replace('L','R')
  img_Concat_name = img.replace('L','C')
  image_L = cv2.imread(IMG_SIDE_L+'/'+img)
  image_R = cv2.imread(IMG_SIDE_R+'/'+img_R)
  image_h = cv2.hconcat([image_L, image_R])
  cv2.imwrite(IMG_CONCAT+'/'+img_Concat_name, image_h)

In [ ]:

FEATURE_THRESHOLD = 250
SEARCH_RANGE = 80
LEFT_INIT_X = 120
RIGHT_INIT_X = 850
SEARCH_WINDOW_SIZE = 11

# 繪圖相關設定
MARK_SIZE = 1
MARK_COLOR = (0, 0, 255)

def normal_distribution(x , mean , sd):
    """
    常態分佈公式
    """
    prob_density = (np.pi*sd) * np.exp(-0.5*((x-mean)/sd)**2)
    return prob_density

def get_normal_distribution_weight(data_len):
    """
    對特定長度的一維資料生成常態分佈的權重
    """
    # Creating a series of data of in range of 0 ~ data_len
    x = np.linspace(0, data_len, data_len)
    # Calculate mean and Standard deviation.
    return normal_distribution(x, np.mean(x), np.std(x))

NORMAL_DISTRIBUTION_WEIGHT = get_normal_distribution_weight(SEARCH_WINDOW_SIZE)

def search_feature_point(img, y, s, e, offset):
    """
    對單張影像的其中一個 Row 搜尋特徵點的座標
    """
    cv2_imshow(img)
    print(y)
    print(s)
    print(e)
    print(offset)
    # 初始化
    max_point_coordinate = (0, 0)
    max_avg_value = 0

    # c_ind : column index (x)
    for x in range(s + offset, e - offset):
        # sliding window (1x5)
        avg_value = np.average(img[y, x-offset : x+offset+1, 0]*NORMAL_DISTRIBUTION_WEIGHT)
        if avg_value > FEATURE_THRESHOLD and avg_value > max_avg_value:
            # print(f"x : {x} , y : {y} => {SEARCH_WINDOW_SIZE} Pixel avg : ", avg_value)
            max_avg_value = avg_value
            max_point_coordinate = (x, y)

    return max_point_coordinate, max_avg_value

def estimate_3D_coordinate(L_uv, R_uv):
    """
    Lecture 8 P.24
    """
    A = np.asarray([
        L_uv[0] * L_P[2] - L_P[0], # u  *  p3' -  p1'
        L_uv[1] * L_P[2] - L_P[1], # v  *  p3' -  p2'
        R_uv[0] * R_P[2] - R_P[0], # up * pp3' - pp1'
        R_uv[1] * R_P[2] - R_P[1]  # vp * pp3' - pp2'
    ])
    U, S, V = np.linalg.svd(A)
    V = V.T
    X = V[:, -1] / V[-1, -1]
    return X

def re_projection(c, Rt, K):
    """
    參考 Lecture 2 P.42 
    x_img = K[R|t] X_world
    將 3D 座標重新投影回影像上
    """
    # World coordinate
    w_c = np.array(c).reshape(4,1)
    # Camera coordinate , Rt : Extrinsic parameter
    c_c = np.dot(Rt, w_c)
    # Scaled image coordinate , K  : Intrinsic parameter
    i_c = np.dot(K, c_c)
    # Image coordinate , 除上第 3 個數值，縮放成影像上的座標
    i_c = np.round(np.true_divide(i_c, i_c[2])).astype(int)
    return i_c[0][0], i_c[1][0]

def rmse(f, c):
    """
    將特徵點座標 ( f ) 與 投影回來的座標 ( c ) 計算 RMSE ( Root Mean Square Error )
    """
    return np.sqrt(
        (
            (f[0] - c[0]) ** 2 +
            (f[1] - c[1]) ** 2
        ) / 2
    )
def main():
    """
    讀取 ./SidebySide 內的 SBS 影像
    用 Filter 找出每個 Row 的特徵點 ( 2D 座標 )
    透過相機參數將 2D 座標投影到 3D 空間 參考 Lecture08 講義 P.23 ~ P.24
    """
    prev_L_avg_x = LEFT_INIT_X
    prev_R_avg_x = RIGHT_INIT_X
    half_search_range = int(SEARCH_RANGE/2)
    offset = int(SEARCH_WINDOW_SIZE/2)
    reconstruction = []

    for i in os.listdir(IMG_CONCAT):
        print(f"Processing : {IMG_CONCAT +'/'+ i}")
        img = cv2.imread(IMG_CONCAT +'/'+ i) # 讀取原始影像
        h, w, c = img.shape
        img_vis = img.copy() # 用來標記找到的特徵點
        cv2_imshow(img)

        # s : search range "start" , e : search range "end"
        L_s = prev_L_avg_x - half_search_range 
        L_e = prev_L_avg_x + half_search_range
        R_s = prev_R_avg_x - half_search_range 
        R_e = prev_R_avg_x + half_search_range

        # 將搜尋範圍做可視化
        img_vis = cv2.line(img_vis, (L_s, 0), (L_s, h), MARK_COLOR, 1)
        img_vis = cv2.line(img_vis, (L_e, 0), (L_e, h), MARK_COLOR, 1)
        img_vis = cv2.line(img_vis, (R_s, 0), (R_s, h), MARK_COLOR, 1)
        img_vis = cv2.line(img_vis, (R_e, 0), (R_e, h), MARK_COLOR, 1)

        # 蒐集特徵點:
        # 1. 用於計算 x 的平均，判斷下一個時間點的前景搜尋範圍
        # 2. 後續以 F_M 確認左右影像的對應關係
        L_features = []
        R_features = []

        # row index (y) 
        for y in range(img.shape[0]):
            # 左半邊影像從 x = 540 以後開始找，避免抓到背景的特徵點
            left_max_c, left_max_v = search_feature_point(img, y, L_s, L_e, offset)
            if left_max_v > 0:
                img_vis = cv2.circle(img_vis, left_max_c, radius=MARK_SIZE, color=MARK_COLOR, thickness=-1) # 標註特徵點
                L_features.append(left_max_c)

            # 右半邊影像從 x = 1800 以後開始找，避免抓到背景的特徵點
            right_max_c, right_max_v = search_feature_point(img, y, R_s, R_e, offset)
            if right_max_v > 0:
                img_vis = cv2.circle(img_vis, right_max_c, radius=MARK_SIZE, color=MARK_COLOR, thickness=-1)# 標註特徵點
                R_features.append(right_max_c)

        # 參考 Lecture 06 P.20 利用 F_M 找兩張影像的對應關係 
        for L_f in L_features:
            x = np.asarray([ # 講義上用的 notation : x
                [L_f[0]], # 座標點的 x
                [L_f[1]], # 座標點的 y
                [1]
            ])
            Fx = np.dot(F_M, x)
            tmp = []

            for R_f in R_features:
                xpT = np.asarray([R_f[0], R_f[1], 1])# 講義上用的 notation : x' Transpose
                tmp.append(np.absolute(np.dot(xpT, Fx))[0]) # 要接近 0
            matched_R_f = R_features[tmp.index(min(tmp))]

            X = estimate_3D_coordinate(L_f, matched_R_f) # Lecture 8 P.24

            # 參考 Lecture 2 P.42 計算 Re-projection error
            L_c = re_projection(X, L_RT, L_K) # Left coordinate
            R_c = re_projection(X, R_RT, R_K) # Right coordinate
            error = round(rmse(L_f, L_c) + rmse(matched_R_f, R_c), 2)
            if error < 10: # 設定閥值過濾偏移太遠的點
                reconstruction.append(np.round(X[0:-1]))

        # 對 x 取平均，並對下個時間點的 x 做 search range 檢核，避免取到背景的光束
        L_x = [xy[0] for xy in L_features]
        R_x = [xy[0] for xy in R_features]
        prev_L_avg_x = round(sum(L_x) / len(L_x))
        prev_R_avg_x = round(sum(R_x) / len(R_x))
        cv2.imwrite(VIS_DIR + i , img_vis) # 將標註結果另存在 feature_points_visualization

    print("Save reconstruction points to .xyz ...")

    # 轉存為 xyz 檔
    f = open(f"./M11015Q03.xyz", "w")
    for p in reconstruction:
        f.write(f"{p[0]} {p[1]} {p[2]}\n")
    f.close()
    print("Done.")



In [ ]:
if __name__ == "__main__":
    main()